Project by Jeremy Bouhi & Lucas Trevalinet

# Classification Document


## Imports

In [1]:
import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET
from pathlib import Path
import re
import pyprind

## STEP 1 : Grab Data

In [3]:
# We consider you had already unzip the dataset file and that this file is in the parent folder

#GLOBAL_PATH = os.path.join("Freemium_cass_global_20180315-170000","20180315-170000", "juri","cass","global") #Lucas' Path
GLOBAL_PATH = os.path.join("..","20180315-170000", "juri","cass","global") #Jeremy's Path

# We need to do that for excluding data from Juri_path
CIVILE_PATH = os.path.join(GLOBAL_PATH,"civile")
COMMERCIALE_PATH = os.path.join(GLOBAL_PATH,"commerciale")
CRIMINELLE_PATH = os.path.join(GLOBAL_PATH,"criminelle")
SOCIALE_PATH = os.path.join(GLOBAL_PATH,"sociale")

In [4]:
from nltk.tokenize import sent_tokenize

ident = []
text = []
division = []
#DATA_PATH = [SOCIALE_PATH] # For faster test
DATA_PATH = [CIVILE_PATH, COMMERCIALE_PATH, CRIMINELLE_PATH, SOCIALE_PATH]

for DIVISION_PATH in DATA_PATH :
    xml_files = list(Path(DIVISION_PATH).glob('**/*.xml'))
    
    for xml_file in xml_files:
    
        with open(xml_file, 'r', encoding="utf-8") as content:

            etree = ET.parse(content) #create an ElementTree object 
            root = etree.getroot()
            
            # For getting the ID
            for child in root.iter('META_COMMUN'):
                id = child.find('ID').text
                ident.append(id)

            
            for child in root.iter('BLOC_TEXTUEL'):
                contenu = "".join(child.itertext())
                text.append(contenu)
                
            # For getting the division    
            for child in root.iter('META_JURI_JUDI'):
                formation = re.sub('CHAMBRE|_|[0-9]', '', child.find('FORMATION').text)
                division.append(formation)


## STEP 2 : Create DataFrame to manipulate easily data

In [21]:
d = {'id': ident, 'text': text, 'division': division}
df = pd.DataFrame(data = d)

# save for the next time
df.to_pickle('3FD8KA7.pkl')
    

In [15]:
print(df[df['text']==""])

Empty DataFrame
Columns: [id, text, division]
Index: []


In [12]:
df = pd.read_pickle('3FD8KA7.pkl')

In [15]:
# remove all empty texts
df = df[df['text']!=""]

In [16]:
fast_execution = True #to run code faster

if(fast_execution) : 
    df = df.sample(500)
    print('You chose to run using fast_execution')
    print('data size: ', df.shape[0])

else :
    print('You chose to run with the whole dataset')
    print('data size: ', df.shape[0])


#formatted_X = X

You chose to run using fast_execution
data size:  500


## STEP 4 : Clean Data

In [17]:
# Preprocessing step pull off all the htlm tag, "\n", and any special characters
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) # remove HTML tags
    text = re.sub('[,\/#!$%\^&\*:{}=\_`~()«»°–]','', text) # remove special characters except ; and . for spliting in sentences
    text = text.replace('\n','').replace('\t','').replace('\'',' ')
    text = re.sub(r'\s{2,}', ' ', text) # remove extra space
    return text

df['text'] = df['text'].apply(preprocessor)
#X = X.apply(preprocessor) # Used for a better display of the most influential sentences
#formatted_X = formatted_X.apply(preprocessor)

In [18]:
# X will be used for cleaning data as asked

X = df['text']
y = df['division']

In [19]:
# The '.'' from lews articles (for example article L. 1256) are bothering for seperate our text in sentences
def clean_references_to_law_articles(text) :
    text = re.sub(r'((?:\b[A-Z][\.-]? ?)?(?:\d+-?\d+\b))', 'N', text)
    return text

In [20]:
from nltk.tokenize import sent_tokenize

# This function is for X. We want the same cleaning as for df['text'] but without spliting in sentences
def clean_before_tokenize(text) :
    text = clean_references_to_law_articles(text)
    text = re.sub('\.\.\.','', text) # remove ...
    text = re.sub(r'\bM\. \b','M ', text) # explicit replacement of M. (because of the .)
    text = text.replace(' ;','. ') # some sentences are split by ;
    return text

def sent_tokenizer(text) : 
    text = clean_before_tokenize(text)
    text = sent_tokenize(text)
    return text

In [21]:
X = X.apply(clean_before_tokenize)
df['text'] = df['text'].apply(sent_tokenizer)

In [22]:
df['text'][20778]

KeyError: 20778

In [12]:
def clean_references_to_law_codes(text) : 
    return re.sub('(?<=code )civil|de l action sociale et des familles|de l artisanat|des assurances|de l aviation civile|du cinéma et de l image animée|de commerce|des communes( de la Nouvelle-Calédonie)?|de la consommation', '', text)

X = X.apply(clean_references_to_law_codes)
#formatted_X = formatted_X.apply(clean_references_to_law_codes)

In [13]:
X = X.apply(clean_references_to_law_articles)

In [14]:
# Remove all numbers

def clean_references_to_law_articles(text) :
    text = re.sub(r'[0-9]*', '', text) #remove digits
    return text

X = X.apply(clean_references_to_law_articles)
#formatted_X = formatted_X.apply(clean_references_to_law_articles)

In [15]:
# Set to lower case
def to_lower_case(s) :
    return s.lower()

X = X.apply(to_lower_case)
#formatted_X = formatted_X.apply(to_lower_case)

In [16]:
# We decided to replace any corresponding labels to a unique special label : *_* 
def clean_words_corresponding_to_labels(text) :
    return re.sub(r'\b(criminel(le)?|commercial(e)?|social(e)?|civil(e))\b', '*_*', text)

X = X.apply(clean_words_corresponding_to_labels)
#formatted_X = formatted_X.apply(clean_words_corresponding_to_labels)

In [17]:
#Stop_words :
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def remove_stop_words(text):
    
    french_stopwords = set(stopwords.words('french'))
    tokens = word_tokenize(text, language='french')
    
    content_tokens = ""
    for token in tokens:
        if token not in french_stopwords:
            content_tokens += token
            content_tokens += " "
    return(content_tokens) 

X = X.apply(remove_stop_words)
#formatted_X = formatted_X.apply(remove_stop_words)

In [18]:
X

15825     attendu arrêt mars cour appel abidjan a condam...
108288    moyen uniquevu les articles decret decembre er...
83130     cassation partielle pourvoi x rene partie *_* ...
131639    moyen additionnel prealable pris violation art...
110736    vu connexité joint les pourvois n.n n.n . prem...
35480     les conclusions fonds garantie demande mise ho...
45541     premier moyenattendu reproche a arret attaque ...
14092     premier moyen pris quatre branchesattendu selo...
126224    vu connexite joint les pourvois caisse primair...
81400     rejet pourvoi taldire cassation arrêt rendu fé...
84502     cass p les pourvois dame x odile x jean contre...
58006     moyen uniqueattendu les dispositions articles ...
18119     moyen unique pris deux branchesattendu société...
83491     rejet pourvoi regulierement forme x jacques co...
107844    premier moyen pris violation article alinea co...
50121     premier moyenattendu ressort enonciations arre...
102661    constate desistement formule a

In [205]:
def clean_data(text):
    #text = preprocessor(text)
    text = to_lower_case(text)
    text = clean_references_to_law_codes(text)
    text = clean_words_corresponding_to_labels(text)
    text = clean_references_to_law_articles(text)
    text = remove_stop_words(text)
    return text


In [204]:
df['text'] = df['text'].apply(preprocessor)
df_clean = df
df_['text'] = df_clean['text'].apply(clean_data)

,id,text,division
130882,JURITEXT000006963337,sur le premier moyenvu le paragraphe 4 de l ar...,SOCIALE
122493,JURITEXT000006990340,sur le moyen unique pris en sa premiere branc...,SOCIALE
8766,JURITEXT000007013250,sur le premier moyenvu l article 845 alinea 6 ...,CIVILE
52801,JURITEXT000006988334,sur le premier moyenvu les articles 6 de l ord...,CIVILE
50351,JURITEXT000006988167,sur le premier moyen pris en sa premiere branc...,CIVILE
130841,JURITEXT000006961809,sur le moyen uniqueattendu que bouteiller prop...,SOCIALE
87701,JURITEXT000007070757,rejet du pourvoi formé par - x... mahamadou co...,CRIMINELLE
82530,JURITEXT000007053013,cassation sur les pourvois de1 x... antonio 2 ...,CRIMINELLE
81401,JURITEXT000007053499,rejet sur les pourvois formes par1 x... eugene...,CRIMINELLE
104164,JURITEXT000007011041,sur le moyen unique pris de la violation de l ...,SOCIALE


## Most influential words

In [19]:
X_civile = X.where(y=='CIVILE')
X_commerciale = X.where(y=='COMMERCIALE')
X_criminelle = X.where(y=='CRIMINELLE')
X_sociale = X.where(y=='SOCIALE')

#to remove NaN values
X_civile = X_civile[~X_civile.isnull()]
X_commerciale = X_commerciale[~X_commerciale.isnull()]
X_criminelle = X_criminelle[~X_criminelle.isnull()]
X_sociale = X_sociale[~X_sociale.isnull()]

In [20]:
#so they all have differents shapes
print(X_civile.shape[0])
print(X_commerciale.shape[0])
print(X_criminelle.shape[0])
print(X_sociale.shape[0])

218
57
97
128


In [21]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

def get_tfidf_matrix(X) : 
    
    count_vect = CountVectorizer()
    freq_term_matrix = count_vect.fit_transform(X)

    tfidf = TfidfTransformer(norm="l2")
    doc_tfidf_matrix = tfidf.fit_transform(freq_term_matrix)
    
    return (doc_tfidf_matrix, count_vect)

In [22]:
def get_influential_words(X, top=10) :
    
    tfidf_matrix, vec = get_tfidf_matrix(X)
    
    tfidf_sorting = np.argsort(tfidf_matrix.toarray()).flatten()[::-1]
    feature_array = np.array(vec.get_feature_names())
    top_n = feature_array[tfidf_sorting][:top]
    
    return top_n

In [23]:
top_civile = get_influential_words(X_civile)
top_commerciale = get_influential_words(X_commerciale)
top_criminelle = get_influential_words(X_criminelle)
top_sociale = get_influential_words(X_sociale)

In [24]:
per_division = {'civile': top_civile, 'commerciale': top_commerciale, 'criminelle': top_criminelle, 'sociale': top_sociale}
words_df = pd.DataFrame(data = per_division)

words_df

,civile,commerciale,criminelle,sociale
0,emi,penale,militaire,régime
1,francs,faillite,gendarme,complémentaire
2,compensation,charblanc,metz,obligatoire
3,déféré,procedure,tribunal,affilié
4,condamnation,surseoir,instruction,géré
5,france,infractions,involontaire,retraite
6,versailles,supporter,forces,france
7,résultant,pouvaient,homicide,médecins
8,celle,influence,armees,membre
9,appel,etait,maniere,volontaire


## Build document summarizer

Once you'll build the tf-idf matrix for the text corpus, you will use the tf-idf of each word to compute a value for each sentence. The n top sentences for a document will be used to represent the document.

In [28]:
#if fast_execution was chosen, find the 1st right index of X (because we only took a sample)

index = 0
while (index not in X) :
    index += 1
print(index)

64


In [33]:
X_tokenized = X.apply(sent_tokenize)
tfidf_matrix, vec = get_tfidf_matrix(X_tokenized[index])

In [37]:
def get_top_sentences_for_document(formatted_X, real_X, index, top=5) :

    top_sentences = []
    sentence_score = []
    tfidf_matrix, vec = get_tfidf_matrix(formatted_X[index])

    for i in range(tfidf_matrix.shape[0]) :
        mat = tfidf_matrix[i].toarray()

        # The score of the sentence corresponds to the sum of the tf-idf of each word / number of words
        sentence_score.append(np.sum(mat) / len(mat[mat > 0]))

    print(sentence_score) 
    top_sentence_score = np.argsort(sentence_score)[::-1][:top]
    print(top_sentence_score)

    for i in range(top) : 
        top_sentences.append(real_X[index][top_sentence_score[i]])
    

    return top_sentences

In [38]:
top_sentences = get_top_sentences_for_document(X_tokenized, df['text'], index)

[0.23564853866238072, 0.2440378798057442, 0.23046054880003009, 0.15608035690034025, 0.23402267453110437, 0.2633222734553934, 0.30720589414548927, 0.4963273338667177, 0.163179436526527, 0.14018661905970106, 0.1695594251072747, 0.2265649047633641, 0.2102962353870599, 0.2962125094107802, 0.36026861504510094, 0.24218547489920036, 0.16787758515588838, 0.1317979180885899, 0.19374058187990234, 0.22195862316836773, 0.12986464043511234, 0.14427330931248036, 0.28122857089799885, 0.23593233428218752, 0.12506348720213756, 0.17106409469569125, 0.16278079925108577, 0.2012808568683767, 0.23563162073691798, 0.30841584340763756, 0.24951822765094583, 0.23342836845070067, 0.23099781304245268, 0.14114886505074764, 0.1636965502677691, 0.14410654680653612, 0.7016482315406753, 0.17576715626983522, 0.2067697262484879, 0.1802779289496648, 0.15890708202069395, 0.195181439761928, 0.27540952765371546, 0.24291318781820015, 0.21369474474972583, 0.16748522902913238, 0.28009898186875243, 0.1994919532399169, 0.1602751

In [39]:
pd.DataFrame(data = {'top sentences of the document' : top_sentences})

,top sentences of the document
0,qu en retenant au contraire pour valider le re...
1,que le N septembre N elle a transféré son sièg...
2,Vu l article N du code de procédure civile rej...
3,Les exonérations sont d application stricte ca...
4,que le 1er septembre N elle a fusionné avec la...


In [41]:
# To display entirely
print('number of sentences of this document: ', len(X_tokenized[index]))
print('top sentences: ', top_sentences)

number of sentences of this document:  49
top sentences:  ['qu en retenant au contraire pour valider le redressement quela Société CEJIP Sécurité qui a déjà bénéficié de l exonération litigieuse ne pouvait plus bénéficier pour les années N et N du dispositif de l assujettissement progressifla cour d appel a dénaturé les termes du litige en violation de l article 4 du code de procédure civile.', 'que le N septembre N elle a transféré son siège social à Aubagne.', 'Vu l article N du code de procédure civile rejette les demandes.', 'Les exonérations sont d application stricte car elles constituent des exceptions à la règle commune.', 'que le 1er septembre N elle a fusionné avec la société Cejip MSI pour prendre la forme en décembre N d une SAS.']


## STEP 3 : Split Data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.2 ,random_state=42,stratify=df["division"] )
print(X_train.head())

## STEP 5 : Train the model

In [21]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [22]:
rnd_clf = RandomForestClassifier(random_state=42, n_estimators=100)
rnd_clf.fit(X_train, y_train)
y_pred = rnd_clf.predict(X_test)

ValueError: could not convert string to float: "\nSur le moyen unique du pourvoi incident, qui est préalable :\n   Attendu, selon l'arrêt attaqué qu'un tribunal de grande instance a prononcé la séparation de corps des époux X... aux torts de l'épouse ; que, par assignation du 21 juillet 1989, le mari a assigné sa femme aux fins de conversion de la séparation de corps en divorce ; que, par assignation en date du 2 août 1989, Mme X..., autorisée par une ordonnance de non-conciliation du 19 janvier 1989, a assigné son mari en divorce pour faute ; qu'après jonction de ces deux procédures, le tribunal de grande instance a prononcé la conversion de la séparation de corps en divorce aux torts de la femme, et, accueillant la demande en divorce de celle-ci, prononcé également le divorce à son profit ; qu'ensuite le Tribunal a constaté que les époux étaient ainsi divorcés aux torts partagés ;\n   Attendu qu'il est fait grief à l'arrêt infirmatif attaqué d'avoir déclaré irrecevable la demande en 

In [153]:
rnd_clf = RandomForestClassifier(random_state=42, n_estimators=100)

ada_clf= AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=100,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)

gdb_clf = GradientBoostingClassifier(random_state=42, n_estimators=100)

voting_clf = VotingClassifier(
    estimators=[('rfc', rnd_clf), ('abc', ada_clf), ('gbr', gdb_clf)],
    voting='hard')

In [154]:
for clf in (rnd_clf, ada_clf, gdb_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

ValueError: could not convert string to float: 'sur le moyen unique du pourvoi incident qui est préalable    attendu selon l arrêt attaqué qu un tribunal de grande instance a prononcé la séparation de corps des époux x aux torts de l épouse  que par assignation du 21 juillet 1989 le mari a assigné sa femme aux fins de conversion de la séparation de corps en divorce  que par assignation en date du 2 août 1989 mme x autorisée par une ordonnance de nonconciliation du 19 janvier 1989 a assigné son mari en divorce pour faute  qu après jonction de ces deux procédures le tribunal de grande instance a prononcé la conversion de la séparation de corps en divorce aux torts de la femme et accueillant la demande en divorce de celleci prononcé également le divorce à son profit  qu ensuite le tribunal a constaté que les époux étaient ainsi divorcés aux torts partagés    attendu qu il est fait grief à l arrêt infirmatif attaqué d avoir déclaré irrecevable la demande en divorce de la femme alors que d 